# Predict and Save Emotion Labels for imdb-wiki

In [ ]:
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 4.0 #was 2.0
    return x

In [ ]:
import cv2
from tensorflow.keras.models import load_model

In [ ]:
data_dir = 'data/' #dir
dataset = pd.read_csv(data_dir+'meta.csv')
image_names = dataset['path']
image_paths = dataset['path'].apply(lambda x: data_dir+x)
image_paths[0]

In [ ]:
# Test Image
img = cv2.imread(image_paths[3])
plt.imshow(img, cmap = plt.get_cmap('gray'))

In [ ]:
# Model Paths
fer = '../models/weights/emotion/emotion_big_XCEPTION.28-0.64.hdf5' #dir
fer_plus = '../models/weights/emotion/plus_emotion_mini_XCEPTION.49-0.78.hdf5' #dir

# Pick Model to Load
emotion_classifier = load_model(fer_plus,compile=False)

In [ ]:
imdb_wiki_preds = []
emotion_target_size = emotion_classifier.input_shape[1:3]
count = 1
count5k = 0
for path in image_paths:
    bgr_image = cv2.imread(path)
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    gray_face = cv2.resize(gray_image, (emotion_target_size))
    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
    imdb_wiki_preds.append(emotion_label_arg)
    if count%5000 == 0:
        count1k+=1
        print('Predicted {} Emotions'.format(count5k*5000))
    count+=1

dataset['plus_emotion'] = imdb_wiki_preds
dataset.to_csv('../data/meta_with_emotion.csv')
print('Labels saved to CSV.')